<a href="https://colab.research.google.com/github/Pulsar-kkaturi/DL-Education/blob/master/VisionDL_Lecture/Lecture5_ResultAnalysis_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Result Visualization (결과 시각화)


# 1. Library Import


In [ ]:
import os, matplotlib, csv, shutil, json
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from IPython.display import Image
from tqdm import tqdm

import skimage
from skimage import transform as skit
from skimage import filters as skif

### PyTorch ###
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

import sklearn.metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
import seaborn as sns

# CUDA 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


# 2. 데이터셋 로딩


In [ ]:
# CIFAR-10 데이터셋 로딩 (PyTorch 방식)
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# NumPy 배열로 변환
x_train = (train_dataset.data.astype(np.float32) / 255.0)
y_train = np.array(train_dataset.targets)
x_test = (test_dataset.data.astype(np.float32) / 255.0)
y_test = np.array(test_dataset.targets)


In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


# 2.1 Cifar10 to Cifar2 (airplane, automobile)


In [ ]:
x_train_list, y_train_list = [], []
x_test_list, y_test_list = [], []
n0, n1 = 0, 0
t0, t1 = 0, 0

for i, i_ in enumerate(x_train):
    if y_train[i] == 0 and n0 < 500:
        arr = skit.resize(i_, (64, 64), anti_aliasing=True)
        onehot = [1, 0]
        n0 += 1
        x_train_list.append(arr)
        y_train_list.append(onehot)
    elif y_train[i] == 1 and n1 < 500:
        arr = skit.resize(i_, (64, 64), anti_aliasing=True)
        onehot = [0, 1]
        n1 += 1
        x_train_list.append(arr)
        y_train_list.append(onehot)

for i, i_ in enumerate(x_test):
    if y_test[i] == 0 and t0 < 100:
        arr = skit.resize(i_, (64, 64), anti_aliasing=True)
        onehot = [1, 0]
        t0 += 1
        x_test_list.append(arr)
        y_test_list.append(onehot)
    elif y_test[i] == 1 and t1 < 100:
        arr = skit.resize(i_, (64, 64), anti_aliasing=True)
        onehot = [0, 1]
        t1 += 1
        x_test_list.append(arr)
        y_test_list.append(onehot)

train_x = np.array(x_train_list)
train_y = np.array(y_train_list)
print(train_x.shape, train_y.shape)
test_x = np.array(x_test_list)
test_y = np.array(y_test_list)
print(test_x.shape, test_y.shape)

# PyTorch tensor로 변환 (channels first 형태로)
train_x_tensor = torch.FloatTensor(train_x).permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
train_y_tensor = torch.FloatTensor(train_y)
test_x_tensor = torch.FloatTensor(test_x).permute(0, 3, 1, 2)
test_y_tensor = torch.FloatTensor(test_y)

print(f"PyTorch tensor shapes:")
print(f"train_x_tensor: {train_x_tensor.shape}, train_y_tensor: {train_y_tensor.shape}")
print(f"test_x_tensor: {test_x_tensor.shape}, test_y_tensor: {test_y_tensor.shape}")


## 2.2 Cifar2 figure


In [ ]:
label_list = ['ariplane', 'automobile']
plt.figure(figsize=(10,10))
num = 3
for i in range(num):
    for j in range(num):
        id = (num*i) + j
        plt.subplot(num,num,id+1)
        plt.imshow(train_x[id])
        plt.title('Class = {}'.format(label_list[list(train_y[id]).index(1)]))


# 3. VGG Model Build


In [ ]:
class ConvBlock2D(nn.Module):
    def __init__(self, in_channels, out_channels, conv_size, num_layers, pool_size, conv_act='relu'):
        super(ConvBlock2D, self).__init__()
        layers = []
        
        for i in range(num_layers):
            if i == 0:
                layers.append(nn.Conv2d(in_channels, out_channels, conv_size, padding=1))
            else:
                layers.append(nn.Conv2d(out_channels, out_channels, conv_size, padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            if conv_act == 'relu':
                layers.append(nn.ReLU(inplace=True))
        
        layers.append(nn.MaxPool2d(kernel_size=pool_size, stride=pool_size))
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)

class OutputBlock(nn.Module):
    def __init__(self, in_features, dens_count, dens_act='relu', drop_rate=0.5, output_count=2):
        super(OutputBlock, self).__init__()
        layers = []
        
        prev_features = in_features
        for i, features in enumerate(dens_count):
            layers.append(nn.Linear(prev_features, features))
            if dens_act == 'relu':
                layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(drop_rate))
            prev_features = features
        
        layers.append(nn.Linear(prev_features, output_count))
        self.classifier = nn.Sequential(*layers)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        return self.classifier(x)


In [ ]:
class VGG16_2D(nn.Module):
    def __init__(self, par_dic):
        super(VGG16_2D, self).__init__()
        
        # Parameters
        self.input_size = par_dic['input_size']
        self.channels = par_dic['channels']
        conv_size = par_dic['conv_size']
        conv_act = par_dic['conv_act']
        pool_size = par_dic['pool_size']
        dens_num = par_dic['dens_num']
        dens_count = par_dic['dens_count']
        dens_act = par_dic['dens_act']
        drop_out = par_dic['drop_out']
        output_count = par_dic['output_count']
        
        # Feature extraction layers
        self.block1 = ConvBlock2D(self.channels, 64, conv_size, 2, pool_size, conv_act)
        self.block2 = ConvBlock2D(64, 128, conv_size, 2, pool_size, conv_act)
        self.block3 = ConvBlock2D(128, 256, conv_size, 3, pool_size, conv_act)
        self.block4 = ConvBlock2D(256, 512, conv_size, 3, pool_size, conv_act)
        self.block5 = ConvBlock2D(512, 512, conv_size, 3, pool_size, conv_act)
        
        # Calculate the size after convolutions for the classifier
        with torch.no_grad():
            dummy_input = torch.zeros(1, self.channels, self.input_size, self.input_size)
            dummy_features = self._forward_features(dummy_input)
            flattened_size = dummy_features.view(1, -1).size(1)
        
        # Classifier
        self.classifier = OutputBlock(flattened_size, dens_count, dens_act, drop_out, output_count)
        
        # Store feature names for Grad-CAM
        self.feature_layers = {
            'block1': self.block1,
            'block2': self.block2,
            'block3': self.block3,
            'block4': self.block4,
            'block5': self.block5
        }
    
    def _forward_features(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        return x
    
    def forward(self, x):
        features = self._forward_features(x)
        output = self.classifier(features)
        return output


In [ ]:
class VGG19_2D(nn.Module):
    def __init__(self, par_dic):
        super(VGG19_2D, self).__init__()
        
        # Parameters
        self.input_size = par_dic['input_size']
        self.channels = par_dic['channels']
        conv_size = par_dic['conv_size']
        conv_act = par_dic['conv_act']
        pool_size = par_dic['pool_size']
        dens_num = par_dic['dens_num']
        dens_count = par_dic['dens_count']
        dens_act = par_dic['dens_act']
        drop_out = par_dic['drop_out']
        output_count = par_dic['output_count']
        
        # Feature extraction layers (VGG19 has more layers)
        self.block1 = ConvBlock2D(self.channels, 64, conv_size, 2, pool_size, conv_act)
        self.block2 = ConvBlock2D(64, 128, conv_size, 2, pool_size, conv_act)
        self.block3 = ConvBlock2D(128, 256, conv_size, 4, pool_size, conv_act)
        self.block4 = ConvBlock2D(256, 512, conv_size, 4, pool_size, conv_act)
        self.block5 = ConvBlock2D(512, 512, conv_size, 4, pool_size, conv_act)
        
        # Calculate the size after convolutions for the classifier
        with torch.no_grad():
            dummy_input = torch.zeros(1, self.channels, self.input_size, self.input_size)
            dummy_features = self._forward_features(dummy_input)
            flattened_size = dummy_features.view(1, -1).size(1)
        
        # Classifier
        self.classifier = OutputBlock(flattened_size, dens_count, dens_act, drop_out, output_count)
        
        # Store feature names for Grad-CAM
        self.feature_layers = {
            'block1': self.block1,
            'block2': self.block2,
            'block3': self.block3,
            'block4': self.block4,
            'block5': self.block5
        }
    
    def _forward_features(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        return x
    
    def forward(self, x):
        features = self._forward_features(x)
        output = self.classifier(features)
        return output


In [ ]:
network_param_set = {'input_size': 64,
                     'channels': 3,
                     'conv_size': 3,
                     'conv_act': 'relu',
                     'pool_size': 2,
                     'dens_num': 2,
                     'dens_count': [1000,500],
                     'dens_act': 'relu',
                     'drop_out': 0.5,
                     'output_count': 2,
                     'output_act': 'softmax'}


In [ ]:
model = VGG16_2D(network_param_set).to(device)
print(model)
print(f"\nModel parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")


# 4. CIFAR2 Training


In [ ]:
# Loss function과 optimizer 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# 데이터 로더 생성
train_dataset = TensorDataset(train_x_tensor, train_y_tensor)
test_dataset = TensorDataset(test_x_tensor, test_y_tensor)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)


In [ ]:
# 학습 함수
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Convert one-hot to class indices
        target_indices = torch.argmax(target, dim=1)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target_indices)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target_indices.view_as(pred)).sum().item()
        total += target.size(0)
    
    return total_loss / len(train_loader), 100. * correct / total

# 검증 함수
def validate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Convert one-hot to class indices
            target_indices = torch.argmax(target, dim=1)
            
            output = model(data)
            loss = criterion(output, target_indices)
            
            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target_indices.view_as(pred)).sum().item()
            total += target.size(0)
    
    return total_loss / len(test_loader), 100. * correct / total


In [ ]:
# 학습 루프
num_epochs = 50
best_val_loss = float('inf')
patience = 10
patience_counter = 0

# History 저장용
history = {
    'loss': [],
    'accuracy': [],
    'val_loss': [],
    'val_accuracy': []
}

for epoch in range(num_epochs):
    # 학습
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 검증
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    
    # Learning rate scheduler
    scheduler.step(val_loss)
    
    # History 저장
    history['loss'].append(train_loss)
    history['accuracy'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)
    
    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print(f'  LR: {optimizer.param_groups[0]["lr"]:.6f}')
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # 최고 모델 저장
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

# 최고 모델 로드
model.load_state_dict(torch.load('best_model.pth'))
print('Training completed!')


# 5. Train Result


## 5.1. Loss & Accuracy


In [ ]:
acc = history['accuracy']
val_acc = history['val_accuracy']
loss = history['loss']
val_loss = history['val_loss']
epochs = range(1, len(acc) + 1)


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.tight_layout()
plt.show()


##5.2. Prediction Result


In [ ]:
test1 = test_x[0]
print(test1.shape)
plt.imshow(test1)
plt.title(label_list[list(test_y[0]).index(1)])


In [ ]:
# 예측 수행
model.eval()
with torch.no_grad():
    test_x_tensor_device = test_x_tensor.to(device)
    scores = model(test_x_tensor_device)
    scores = F.softmax(scores, dim=1)  # softmax 적용
    scores = scores.cpu().numpy()  # CPU로 이동 후 numpy 변환

new_scores = []
for score in scores:
    max_val = np.max(score)
    prob_num = label_list[list(score).tolist().index(max_val)]
    new_scores.append(prob_num)
print(new_scores)


In [ ]:
new_labels = []
for y in test_y:
    max_val = np.max(y)
    prob_num = label_list[list(y).index(max_val)]
    new_labels.append(prob_num)
print(new_labels)


In [ ]:
plt.imshow(test_x[0])
print(f'label={label_list[list(test_y[0]).index(1)]}, predict={new_scores[0]}')
print(scores[0])


### 5.3. Confusion Matrix


In [ ]:
conf = confusion_matrix(new_labels, new_scores, labels=label_list)
print(conf)


In [ ]:
plt.figure(figsize=(10,10))
ax = sns.heatmap(conf, annot = True, cmap="coolwarm", vmax = 100,
                 annot_kws={"fontsize":20}, center=50, cbar=True,
                 xticklabels=label_list, yticklabels=label_list)

# labels, title and ticks
ax.set_xlabel('Predicted labels',fontsize=20);ax.set_ylabel('True labels',fontsize=20);
ax.set_title('Confusion Matrix', fontsize=30);
ax.xaxis.set_ticklabels(label_list, fontsize=20); ax.yaxis.set_ticklabels(label_list, fontsize=20)


# 6. Result Visualization


## 6.1. Class Activation Map


먼저 GRAD-CAM을 구해주는 함수를 정의한다.


In [ ]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.feature_maps = None
        self.gradients = None
        
        # Hook 등록
        self.target_layer.register_forward_hook(self.save_feature_maps)
        self.target_layer.register_backward_hook(self.save_gradients)
    
    def save_feature_maps(self, module, input, output):
        self.feature_maps = output
    
    def save_gradients(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]
    
    def generate_cam(self, input_image, class_idx=None):
        # Forward pass
        output = self.model(input_image)
        
        if class_idx is None:
            class_idx = torch.argmax(output, dim=1)
        
        # Backward pass
        self.model.zero_grad()
        output[:, class_idx].backward(retain_graph=True)
        
        # Generate CAM
        gradients = self.gradients[0]  # (C, H, W)
        feature_maps = self.feature_maps[0]  # (C, H, W)
        
        # Global average pooling of gradients
        weights = torch.mean(gradients, dim=(1, 2))  # (C,)
        
        # Weighted combination of feature maps
        cam = torch.zeros(feature_maps.shape[1:], dtype=torch.float32, device=feature_maps.device)
        for i, w in enumerate(weights):
            cam += w * feature_maps[i, :, :]
        
        # Apply ReLU
        cam = F.relu(cam)
        
        # Normalize
        cam = cam - cam.min()
        cam = cam / cam.max()
        
        return cam.cpu().numpy()

def make_gradcam_heatmap(img_array, model, target_layer_name):
    # target_layer 찾기
    target_layer = None
    for name, module in model.named_modules():
        if target_layer_name in name:
            target_layer = module
            break
    
    if target_layer is None:
        print(f"Target layer {target_layer_name} not found!")
        return None
    
    # GradCAM 객체 생성
    grad_cam = GradCAM(model, target_layer)
    
    # CAM 생성
    heatmap = grad_cam.generate_cam(img_array)
    
    return heatmap


In [ ]:
# 테스트 이미지에 대한 히트맵을 구한다.
test1_tensor = test_x_tensor[0:1].to(device)  # 배치 차원 추가
heatmap1 = make_gradcam_heatmap(test1_tensor, model, 'block4')
heatmap2 = make_gradcam_heatmap(test1_tensor, model, 'block5')
print(test1_tensor.shape, heatmap1.shape, heatmap2.shape)

# Display heatmap
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
plt.imshow(heatmap1)
plt.title('Block4 Heatmap')
plt.subplot(1,2,2)
plt.imshow(heatmap2)
plt.title('Block5 Heatmap')


In [ ]:
# 원본크기에 맞게 히트맵 변환
hm1 = skit.resize(heatmap1, (64, 64), anti_aliasing=True)
hm1 = skif.gaussian(hm1, sigma=3)
hm2 = skit.resize(heatmap2, (64, 64), anti_aliasing=True)
hm2 = skif.gaussian(hm2, sigma=3)
hmn = (hm1+hm2)/2

# 히트맵을 0 ~255 사이 값으로 재조정
heatmap_n = np.uint8(255 * hmn)
test_n = np.uint8(255*test_x[0])

# 결과 시각화
plt.figure(figsize=(15,8))
plt.subplot(1,3,1)
plt.imshow(test_n)
plt.title('Original Image')
plt.subplot(1,3,2)
plt.imshow(heatmap_n)
plt.title('Heatmap')
plt.subplot(1,3,3)
plt.imshow(test_n)
plt.imshow(heatmap_n, alpha=0.5)
plt.title('Overlay')


### 6.2. CAM 모듈화


In [ ]:
def make_gradcam(img_arr):
    # NumPy 이미지를 PyTorch tensor로 변환
    if isinstance(img_arr, np.ndarray):
        if len(img_arr.shape) == 3:  # (H, W, C)
            img_tensor = torch.FloatTensor(img_arr).permute(2, 0, 1).unsqueeze(0)  # (1, C, H, W)
        else:  # 이미 (C, H, W) 형태
            img_tensor = torch.FloatTensor(img_arr).unsqueeze(0)
    else:
        img_tensor = img_arr
    
    img_tensor = img_tensor.to(device)
    
    hm1 = make_gradcam_heatmap(img_tensor, model, 'block4')
    hm2 = make_gradcam_heatmap(img_tensor, model, 'block5')

    # 원본크기에 맞게 히트맵 변환
    hm1 = skit.resize(hm1, (64, 64), anti_aliasing=True)
    hm1 = skif.gaussian(hm1, sigma=3)
    hm2 = skit.resize(hm2, (64, 64), anti_aliasing=True)
    hm2 = skif.gaussian(hm2, sigma=3)
    hmn = (hm1+hm2)/2

    # 히트맵을 0 ~255 사이 값으로 재조정
    heatmap1 = np.uint8(255 * hmn)
    
    # 원본 이미지도 0-255 범위로
    if isinstance(img_arr, np.ndarray):
        img1 = np.uint8(255 * img_arr)
    else:
        img1 = np.uint8(255 * img_arr)
    
    return img1, heatmap1


In [ ]:
plt.figure(figsize=(20,20))
n, s = 5, 0
for i in range(n):
    for j in range(n):
        idx = (i*n)+j+s
        if idx >= len(test_x):
            break
        img, hm = make_gradcam(test_x[idx])
        plt.subplot(n,n,(i*n)+j+1)
        plt.imshow(img)
        plt.imshow(hm, alpha=0.5, cmap='Reds')
        if new_scores[idx] == new_labels[idx]:
            cor = 'True'
        else:
            cor = 'False'
        plt.title(f'({cor})')
        plt.axis('off')


## 6.3. Feature Map 시각화


In [ ]:
# Hook을 사용하여 중간 층의 출력을 저장하는 클래스
class FeatureExtractor:
    def __init__(self, model, layers):
        self.model = model
        self.layers = layers
        self.features = {}
        self.hooks = []
        
        for layer_name in layers:
            layer = dict(model.named_modules())[layer_name]
            hook = layer.register_forward_hook(self.get_features(layer_name))
            self.hooks.append(hook)
    
    def get_features(self, name):
        def hook(model, input, output):
            self.features[name] = output.detach()
        return hook
    
    def remove_hooks(self):
        for hook in self.hooks:
            hook.remove()

# 특정 층들의 feature map을 추출
feature_layers = []
for name, module in model.named_modules():
    if isinstance(module, (nn.Conv2d, nn.MaxPool2d)):
        feature_layers.append(name)

print("Available feature layers:")
for layer in feature_layers:
    print(f"  {layer}")

# Feature extractor 생성
extractor = FeatureExtractor(model, feature_layers)


In [ ]:
# 테스트 이미지를 모델에 통과시켜 feature map 추출
test_img_tensor = test_x_tensor[0:1].to(device)
print(test_img_tensor.shape)

# 이미지 표시 (원본)
plt.figure(figsize=(5, 5))
plt.imshow(test_x[0])
plt.title('Input Image')
plt.axis('off')

# Forward pass를 통해 feature map들 추출
with torch.no_grad():
    _ = model(test_img_tensor)

# 추출된 feature map들 확인
print("\nExtracted feature maps:")
for layer_name, features in extractor.features.items():
    print(f"{layer_name}: {features.shape}")


In [ ]:
def show_feature_map(features, all_mode=True):
    if isinstance(features, torch.Tensor):
        features = features.cpu().numpy()
    
    plt.figure(figsize=(10,10))
    if all_mode:
        chn = features.shape[1]  # PyTorch는 (N, C, H, W)
        chns = int(np.ceil(np.sqrt(chn)))
    else:
        chn = min(16, features.shape[1])
        chns = 4
    
    for i in range(chn):
        plt.subplot(chns, chns, i+1)
        plt.imshow(features[0, i, ...], cmap='viridis')  # (N, C, H, W) -> (H, W)
        plt.axis('off')
    plt.tight_layout()


In [ ]:
# 첫 번째 conv layer의 feature map 시각화
first_conv_features = None
for layer_name, features in extractor.features.items():
    if 'conv' in layer_name.lower():
        first_conv_features = features
        print(f"Showing feature maps from: {layer_name}")
        break

if first_conv_features is not None:
    show_feature_map(first_conv_features, all_mode=False)


In [ ]:
# 여러 층의 feature map을 한번에 시각화
plt.figure(figsize=(20, 30))
layer_count = 0

for layer_name, features in extractor.features.items():
    if layer_count >= 10:  # 최대 10개 층만 표시
        break
    
    if isinstance(features, torch.Tensor):
        features_np = features.cpu().numpy()
    else:
        features_np = features
    
    # 각 층에서 첫 4개 채널만 표시
    num_channels = min(4, features_np.shape[1])
    
    for j in range(num_channels):
        plt.subplot(10, 4, (layer_count * 4) + j + 1)
        plt.imshow(features_np[0, j, ...], cmap='viridis')
        plt.title(f'{layer_name}\nCh {j}')
        plt.axis('off')
    
    layer_count += 1

plt.tight_layout()
plt.show()

# Hook 제거
extractor.remove_hooks()


## 6.4. Filter 시각화


In [ ]:
# Filter 시각화를 위한 클래스
class FilterVisualizer:
    def __init__(self, model, layer_name):
        self.model = model
        self.layer_name = layer_name
        self.target_layer = None
        
        # 타겟 레이어 찾기
        for name, module in model.named_modules():
            if name == layer_name:
                self.target_layer = module
                break
        
        if self.target_layer is None:
            raise ValueError(f"Layer {layer_name} not found in model")
    
    def compute_loss(self, image, filter_index):
        # Feature extractor 생성
        feature_extractor = FeatureExtractor(self.model, [self.layer_name])
        
        # Forward pass
        _ = self.model(image)
        
        # 해당 레이어의 출력 가져오기
        activation = feature_extractor.features[self.layer_name]
        
        # 특정 필터의 활성화 평균
        if len(activation.shape) == 4:  # (N, C, H, W)
            filter_activation = activation[:, filter_index, 2:-2, 2:-2]
        else:
            filter_activation = activation[:, filter_index]
        
        loss = torch.mean(filter_activation)
        
        # Hook 제거
        feature_extractor.remove_hooks()
        
        return loss
    
    def gradient_ascent_step(self, image, filter_index, learning_rate):
        image.requires_grad_(True)
        
        loss = self.compute_loss(image, filter_index)
        
        # Backward pass
        loss.backward()
        
        # Gradient ascent
        with torch.no_grad():
            # L2 정규화
            grads = image.grad
            grads = grads / (torch.norm(grads) + 1e-8)
            
            # 업데이트
            image += learning_rate * grads
            
            # 그래디언트 초기화
            image.grad.zero_()
        
        return image
    
    def generate_filter_pattern(self, filter_index, img_width=64, img_height=64, iterations=30, learning_rate=10.0):
        # 랜덤 이미지 생성
        image = torch.rand(1, 3, img_height, img_width, device=device) * 0.2 + 0.4
        
        for i in range(iterations):
            image = self.gradient_ascent_step(image, filter_index, learning_rate)
        
        return image[0].detach().cpu().numpy()
    
    def deprocess_image(self, image, margin=0.1):
        # 채널을 마지막으로 이동: (C, H, W) -> (H, W, C)
        image = np.transpose(image, (1, 2, 0))
        
        # 정규화
        image -= np.mean(image)
        image /= (np.std(image) + 1e-8)
        image *= 64
        image += 128
        image = np.clip(image, 0, 255).astype(np.uint8)
        
        # 마진 제거
        margin_pixels = int(image.shape[0] * margin)
        if margin_pixels > 0:
            image = image[margin_pixels:-margin_pixels, margin_pixels:-margin_pixels, :]
        
        return image
    
    def visualize_filters(self, num_filters=25, width=64, height=64):
        filters_per_row = int(np.sqrt(num_filters))
        
        all_images = []
        for filter_index in tqdm(range(num_filters), desc="Generating filter visualizations"):
            try:
                raw_image = self.generate_filter_pattern(filter_index, width, height)
                processed_image = self.deprocess_image(raw_image, 0.1)
                all_images.append(processed_image)
            except Exception as e:
                print(f"Error generating filter {filter_index}: {e}")
                # 빈 이미지 추가
                all_images.append(np.zeros((height-int(height*0.2), width-int(width*0.2), 3), dtype=np.uint8))
        
        # 시각화
        plt.figure(figsize=(10, 10))
        for i in range(filters_per_row):
            for j in range(filters_per_row):
                idx = i * filters_per_row + j
                if idx < len(all_images):
                    plt.subplot(filters_per_row, filters_per_row, idx + 1)
                    plt.imshow(all_images[idx])
                    plt.axis('off')
                    plt.title(f'Filter {idx}')
        
        plt.tight_layout()
        plt.show()

def filter_visual(layer_name, width=64, height=64):
    try:
        visualizer = FilterVisualizer(model, layer_name)
        visualizer.visualize_filters(25, width, height)
    except Exception as e:
        print(f"Error in filter visualization: {e}")


### 6.4.1. cifar2 result


In [ ]:
# 사용 가능한 Conv2D 레이어들 확인
conv_layers = []
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        conv_layers.append(name)
        print(name)

print(f"\nTotal Conv2D layers found: {len(conv_layers)}")


In [ ]:
# Block5의 첫 번째 conv layer 필터 시각화
if len(conv_layers) > 0:
    # block5의 conv layer 찾기
    block5_conv = None
    for layer_name in conv_layers:
        if 'block5' in layer_name and 'conv' in layer_name:
            block5_conv = layer_name
            break
    
    if block5_conv:
        print(f"Visualizing filters from: {block5_conv}")
        filter_visual(block5_conv, 64, 64)
    else:
        # 대안으로 마지막 conv layer 사용
        print(f"Block5 conv not found, using: {conv_layers[-1]}")
        filter_visual(conv_layers[-1], 64, 64)


### 6.4.2. ImageNet Result


In [ ]:
# ImageNet pretrained VGG16 로드
import torchvision.models as models

vgg16_pretrained = models.vgg16(pretrained=True).to(device)
vgg16_pretrained.eval()

# Conv2D 레이어들 확인
imagenet_conv_layers = []
for name, module in vgg16_pretrained.named_modules():
    if isinstance(module, nn.Conv2d):
        imagenet_conv_layers.append(name)
        print(name)

print(f"\nTotal Conv2D layers in ImageNet VGG16: {len(imagenet_conv_layers)}")


In [ ]:
print(vgg16_pretrained)


In [ ]:
# ImageNet VGG16의 특정 층 필터 시각화
if len(imagenet_conv_layers) > 0:
    # features.28은 대략 block5_conv1에 해당
    target_layer = 'features.28'  # 또는 다른 적절한 레이어
    if target_layer in imagenet_conv_layers:
        print(f"Visualizing ImageNet VGG16 filters from: {target_layer}")
        
        # ImageNet용 filter visualizer
        imagenet_visualizer = FilterVisualizer(vgg16_pretrained, target_layer)
        imagenet_visualizer.visualize_filters(25, 256, 256)
    else:
        print(f"Target layer not found, available layers: {imagenet_conv_layers[:5]}...")  # 처음 5개만 표시
